# Docker and Apptainer: Container Solutions for HPC

## Introduction

Containers package applications and their dependencies into portable, reproducible environments. While Docker is the industry standard for containerization, High Performance Computing (HPC) systems require different solutions due to security and infrastructure constraints.

### Why Apptainer for HPC?

**Docker** requires root privileges and is not permitted on most HPC systems for security reasons. **Apptainer** (formerly Singularity) was designed specifically for HPC environments:

- ✅ Runs without root privileges
- ✅ Integrates with HPC schedulers (SLURM, PBS/Torque, etc.)
- ✅ Compatible with Docker images
- ✅ Optimized for parallel filesystems
- ✅ Supports MPI and GPU workloads

### Container Image Formats

**Docker Images:** Multi-layer, mutable format stored in registries like Docker Hub or GitHub Container Registry (ghcr.io)

**Apptainer Formats:**
- **SIF (Singularity Image Format):** Single compressed, read-only file - ideal for distribution and archiving
- **Sandbox:** Uncompressed directory - ideal for development and systems with resource constraints

### Architecture Considerations

Container images are built for specific CPU architectures:
- **amd64 (x86-64):** Intel/AMD processors - used by most HPC systems
- **arm64:** ARM processors - used by Apple Silicon Macs (M1/M2/M3)

⚠️ **Important:** You cannot run amd64 containers on arm64 systems and vice versa. Most HPC systems are amd64, so containers must be built for that architecture.

### Workflows Covered in This Guide

This guide presents three approaches for creating Apptainer containers:

1. **Building on NEC HPC (Sandbox Method):** Create containers directly on the HPC system using the sandbox format to work around memory constraints
2. **Building on Mac/Linux (SIF Method):** Create portable SIF files on your local machine and transfer them to HPC
3. **Building on GitHub Actions (Automated SIF):** Use GitHub's CI/CD infrastructure to automatically build and distribute SIF files

Choose the workflow that best fits your needs:
- Use **Method 1 (NEC)** if you only have access to the HPC system
- Use **Method 2 (Mac/Linux)** if you want to build containers locally
- Use **Method 3 (GitHub)** for automated builds, version control, and easy distribution to multiple users

# 1
-----

# 1. Running Docker Containers on NEC HPC with Apptainer

This tutorial explains how to pull and run Docker containers on the RCNL (NEC HPC system) using Apptainer.

## Background

The RCNL HPC system uses **Apptainer** (formerly Singularity) instead of Docker. Apptainer can pull Docker images and convert them to a format that works on HPC systems without requiring root privileges.

## Problem: Memory Issues During Container Pull

When pulling large Docker images, Apptainer compresses them into SIF format using squashfs, which is very memory-intensive and can fail on login nodes.

**Error you might see:**
```
FATAL ERROR: Out of memory (frag_thrd)
FATAL ERROR: Failed to create thread
```

## Solution: Use Batch Jobs with Sandbox Format

### Step 1: Create a Job Submission Script

Create a file called `pull_image.sh`:

```bash
#!/bin/bash
#PBS -q gp_norm_all
#PBS -l elapstim_req=01:00:00
#PBS -l cpunum_job=4
#PBS -l memsz_job=64gb
#PBS -N apptainer_pull
#PBS -o apptainer_pull.out
#PBS -e apptainer_pull.err

cd /hpc/uwork/rpotthas/apptainer_test

export APPTAINER_TMPDIR=/hpc/uwork/rpotthas/tmp
export APPTAINER_CACHEDIR=/hpc/uwork/rpotthas/cache
mkdir -p $APPTAINER_TMPDIR
mkdir -p $APPTAINER_CACHEDIR

# Build as sandbox (uncompressed directory)
apptainer build --sandbox docker-plot-03-sandbox docker://ghcr.io/eumetnet-e-ai/docker-plot-03:latest

echo "Build completed at $(date)"
rpotthas@rcnl100:/hpc/uwork/rpotthas/apptainer_test> 
```

**Key parameters:**
- `-q gp_norm_all`: Queue name (supports up to 1920GB RAM)
- `-l memsz_job=128gb`: Request 128GB memory
- `-l cpunum_job=2`: Use 2 CPUs (fewer threads = less memory)
- `--sandbox`: Creates uncompressed directory instead of compressed SIF file

### Step 2: Submit the Job

```bash
qsub pull_image.sh
```

### Step 3: Monitor the Job

```bash
# Check job status
qstat -u rpotthas

# View output in real-time
tail -f apptainer_pull.out
tail -f apptainer_pull.err
```

### Step 4: Verify the Container

Once the job completes, you should see a directory:

```bash
ls -lh docker-plot-03-sandbox/
```

## Running Your Code in the Container

### Basic Execution

```bash
apptainer exec docker-plot-03-sandbox python -B 03_plot_curve.py
```

**Important:** Use the `-B` flag with Python to bypass corrupted bytecode files that sometimes occur in Docker→Apptainer conversions.

```
rpotthas@rcnl100:/hpc/uwork/rpotthas/apptainer_test> ll
total 72
-rw-r-----  1 rpotthas fe1 50662 Jan  3 12:08 03_plot_curve.png
-rw-r-----  1 rpotthas fe1   417 Jan  3 12:06 03_plot_curve.py
-rw-r-----  1 rpotthas fe1  2750 Jan  3 12:07 apptainer_pull.err
-rw-r-----  1 rpotthas fe1   746 Jan  3 12:07 apptainer_pull.out
drwxr-xr-x 19 rpotthas fe1  4096 Jan  3 12:07 docker-plot-03-sandbox
-rw-r-----  1 rpotthas fe1   561 Jan  3 12:03 pull_image.sh
```


### Other Useful Commands

```bash
# Get an interactive shell inside the container
apptainer shell docker-plot-03-sandbox

# Run the container's default command
apptainer run docker-plot-03-sandbox

# Execute any command in the container
apptainer exec docker-plot-03-sandbox <command>
```

## Creating a Convenience Wrapper

Create a script `run-python.sh` for easier use:

```bash
#!/bin/bash
apptainer exec /hpc/uwork/rpotthas/apptainer_test/docker-plot-03-sandbox python -B "$@"
```

Make it executable:

```bash
chmod +x run-python.sh
```

Use it:

```bash
./run-python.sh my_script.py
```

## Understanding Queue Resources

Check available queues and their limits:

```bash
qstat -Qf
```

On RCNL, `gp_norm_all` queue provides:
- **Max Memory:** 1920GB per job
- **Max CPUs:** 128 per job
- **Default Memory:** 15GB (too low for most container builds)

Always explicitly request memory with `-l memsz_job=<size>`.

## Troubleshooting

### Issue: "Out of memory" errors

**Solution:** Request more memory or use sandbox format
```bash
#PBS -l memsz_job=128gb
```

### Issue: Python bytecode corruption (`EOFError: marshal data too short`)

**Solution:** Use Python's `-B` flag to bypass bytecode
```bash
apptainer exec container python -B script.py
```

### Issue: Container build fails repeatedly

**Solution:** Try these steps in order:
1. Increase memory allocation (64GB → 128GB)
2. Reduce CPU count (fewer threads = less memory)
3. Use `--sandbox` instead of pulling to SIF
4. Clear cache: `rm -rf ~/.apptainer/cache`

## Converting Sandbox to SIF (Optional)

**⚠️ IMPORTANT: SIF conversion may not work on RCNL** due to system limits on virtual memory (8GB cap) and process counts. The squashfs compression used to create SIF files requires more resources than the system allows.

You can attempt conversion with this job script:

```bash
#!/bin/bash
#PBS -q gp_norm_all
#PBS -l elapstim_req=02:00:00
#PBS -l cpunum_job=1
#PBS -l memsz_job=512gb
#PBS -N sandbox_to_sif

cd /hpc/uwork/rpotthas/apptainer_test

ulimit -v unlimited 2>/dev/null || echo "Cannot increase virtual memory limit"
ulimit -u 4096 2>/dev/null || echo "Cannot increase process limit"

apptainer build container.sif docker-plot-03-sandbox/

echo "Conversion completed at $(date)"
```

**Note:** For most HPC use cases, **sandboxes are perfectly fine and often preferred**:
- Same runtime performance as SIF
- Easier to debug and modify
- No compression overhead
- Work reliably on systems with strict limits

If SIF conversion consistently fails due to system limits, contact your HPC support team or simply continue using the sandbox format.

## Tips

- **Always use batch jobs** for pulling containers (don't run on login nodes)
- **Request sufficient memory** (128GB is a safe bet for most images)
- **Use sandbox format** to avoid compression memory issues
- **Keep the `-B` flag** when running Python to avoid bytecode issues
- **Check your quota** with `quota` command before large pulls

## Additional Resources

- Apptainer documentation: https://apptainer.org/docs/
- NEC NQSV queue system documentation (check with your system admin)




# 2
-----

# Building SIF Files on Mac using Lima and Apptainer

This guide explains how to build Apptainer SIF files on macOS using Lima (Linux Virtual Machine).

## Why Lima?

Apptainer requires Linux and cannot run natively on macOS. Lima provides a lightweight Linux VM that integrates seamlessly with macOS.

## Prerequisites

- macOS (Intel or Apple Silicon)
- Homebrew installed
- GitHub account with access to container registries

## Step 1: Install Lima

```bash
# Install Lima via Homebrew
brew install lima

# Verify installation
limactl --version
```

## Step 2: Create and Start a Lima VM

```bash
# Start the default Lima instance (Ubuntu)
limactl start

# This will:
# - Download Ubuntu image (~500MB)
# - Create a VM with 4GB RAM and 100GB disk
# - Set up networking and file sharing
# - Take a few minutes on first run

# Check VM status
limactl list
```

## Step 3: Enter the Lima VM

```bash
# SSH into the Lima VM
lima

# You're now in an Ubuntu Linux environment
# Your macOS home directory is mounted at /Users/YOUR_USERNAME
```

## Step 4: Install Apptainer in Lima

Inside the Lima VM:

```bash
# Update package list
sudo apt update

# Add Apptainer repository
sudo add-apt-repository -y ppa:apptainer/ppa

# Install Apptainer
sudo apt install -y apptainer

# Verify installation
apptainer --version
```

Expected output: `apptainer version 1.4.0` (or newer)

## Step 5: Set Up GitHub Container Registry Credentials

### Create a GitHub Personal Access Token (PAT)

1. Go to https://github.com/settings/tokens
2. Click **"Generate new token"** → **"Tokens (classic)"**
3. Give it a name: `apptainer-ghcr-access`
4. Set expiration (recommended: 90 days)
5. Check the **`read:packages`** scope
6. Click **"Generate token"**
7. **Copy the token immediately** (you won't see it again)

### Configure Docker Credentials Permanently in Lima

In Lima, set up credentials that persist across sessions:

```bash
# Create Docker config directory
mkdir -p ~/.docker

# Login to GitHub Container Registry
# This saves credentials to ~/.docker/config.json permanently
docker login ghcr.io -u YOUR_GITHUB_USERNAME
# Password: paste your PAT (not your GitHub password)
```

**Verify credentials are saved:**

```bash
# Check Docker config exists and has credentials
cat ~/.docker/config.json

# Should show something like:
# {
#   "auths": {
#     "ghcr.io": {
#       "auth": "base64_encoded_credentials"
#     }
#   }
# }
```

Now your credentials are permanently stored and will be used automatically for future builds!

### Alternative: Environment Variables (Optional)

If you prefer environment variables, add them to your shell profile:

```bash
# Edit your bash profile
nano ~/.bashrc

# Add these lines at the end:
export APPTAINER_DOCKER_USERNAME=YOUR_GITHUB_USERNAME
export APPTAINER_DOCKER_PASSWORD=YOUR_GITHUB_PAT

# Save and exit (Ctrl+X, Y, Enter)

# Reload the profile
source ~/.bashrc
```

**Note:** The Docker config method is recommended as it's more secure and standard.

## Step 6: Build SIF Files

### Understanding Architecture Compatibility

Container images are built for specific CPU architectures:
- **amd64 (x86-64):** Used by most HPC systems and Intel/AMD servers
- **arm64 (ARM):** Used by Apple Silicon Macs (M1/M2/M3)

You'll need to build different versions depending on where you want to run them:

| Architecture | Where to Use | Can Test on Mac? |
|--------------|--------------|------------------|
| amd64 | HPC/Linux servers (RCNL) | ❌ No (Apple Silicon can't run amd64) |
| arm64 | Apple Silicon Mac | ✅ Yes (for local testing only) |

### Build for HPC (amd64 - Required)

This is the version you'll transfer to RCNL or other HPC systems:

```bash
# Navigate to Lima's home directory (writable)
cd ~

# Build amd64 version for HPC
apptainer build docker-plot-03-amd64.sif docker://ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

You'll see a warning: `WARNING: Architecture amd64 does not match build arch arm64v8` - this is expected and harmless.

### Build for Local Testing (arm64 - Optional)

If you want to test the container on your Mac before transferring to HPC:

```bash
# Build arm64 version for Apple Silicon
apptainer build docker-plot-03-arm64.sif docker://ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

**Note:** If the Docker image doesn't have an arm64 variant, this will fail. In that case, you can only build amd64 and test on the HPC system.

### Expected Build Output

```
INFO:    Starting build...
INFO:    Fetching OCI image...
INFO:    Extracting OCI image...
INFO:    Creating SIF file...
INFO:    Build complete: docker-plot-03-amd64.sif
```

### Understanding the Build Command

The command breaks down as:
- `apptainer build` - Build a container image
- `docker-plot-03-amd64.sif` - Output filename
- `docker://` - Pull from a Docker registry
- `ghcr.io/eumetnet-e-ai/docker-plot-03:latest` - Image location and tag

### Recommended Workflow

**For most users:**
1. Build amd64 version only (for HPC)
2. Transfer directly to HPC
3. Test on HPC system

**If you want local validation:**
1. Build arm64 version for Mac testing
2. Test your scripts locally
3. Build amd64 version for HPC
4. Transfer to HPC

**Important:** Always remember which version is which - use clear naming like `-amd64` and `-arm64` suffixes!

## Step 7: Test the SIF File Locally

Before transferring to HPC, test that the SIF works in Lima:

```bash
# Still in Lima VM, test the container
apptainer exec docker-plot-03.sif python --version

# Run a simple command
apptainer exec docker-plot-03.sif ls /

# Get an interactive shell
apptainer shell docker-plot-03.sif

# Inside the container shell:
whoami
python --version
exit
```

### Test with your actual script

If you have a test script, try running it:

```bash
# Create a simple test script in Lima
cat > test_plot.py << 'EOF'
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

print("Matplotlib version:", matplotlib.__version__)
print("Creating test plot...")

x = np.linspace(0, 10, 100)
y = np.sin(x)

plt.figure(figsize=(8, 6))
plt.plot(x, y)
plt.title("Test Plot")
plt.savefig("test_output.png")
print("Plot saved as test_output.png")
EOF

# Run the script in the container
apptainer exec docker-plot-03.sif python -B test_plot.py

# Check if the output was created
ls -lh test_output.png
```

**Note:** Use the `-B` flag with Python to bypass potential bytecode issues (same as on HPC).

### Verify the SIF is complete

```bash
# Check SIF file info
apptainer inspect docker-plot-03.sif

# Verify size (should be several hundred MB)
ls -lh docker-plot-03.sif
```

## Step 8: Copy SIF to macOS

Exit Lima and use `limactl` from your Mac terminal:

```bash
# Exit Lima VM
exit

# Copy from Lima to Mac
limactl copy default:docker-plot-03.sif ~/Desktop/docker-plot-03.sif

# Or to a specific directory
limactl copy default:docker-plot-03.sif ~/path/to/destination/docker-plot-03.sif
```

Verify the file on macOS:

```bash
ls -lh ~/Desktop/docker-plot-03.sif
```

## Step 8: Transfer to HPC System

```bash
# Transfer to your HPC system
scp ~/Desktop/docker-plot-03.sif username@hpc-system:/path/to/destination/
```

Example for RCNL:
```bash
scp ~/Desktop/docker-plot-03.sif rpotthas@rcnl100:/hpc/uwork/rpotthas/apptainer_test/
```

## Common Issues and Solutions

### Issue: "Read-only file system" when building

**Cause:** macOS directories mounted in Lima are read-only

**Solution:** Always build in Lima's home directory (`~`), not in `/Users/...`

```bash
cd ~  # Build here
apptainer build myimage.sif docker://...
```

### Issue: "Authentication required" error

**Cause:** Container registry requires login

**Solution:** Login with Docker or set environment variables

```bash
# Method 1: Docker login
docker login ghcr.io

# Method 2: Environment variables
export APPTAINER_DOCKER_USERNAME=your_username
export APPTAINER_DOCKER_PASSWORD=your_pat
```

### Issue: "Transport is not supported" with registry login

**Cause:** `apptainer registry login` doesn't work with all registries

**Solution:** Use Docker credentials instead

```bash
# Use docker login instead
docker login ghcr.io
```

### Issue: Architecture warning (amd64 vs arm64)

**Warning:** `Architecture amd64 does not match build arch arm64v8`

**Impact:** This is just a warning. The SIF will work on Linux amd64 systems (most HPC clusters)

**Explanation:** You're building on Apple Silicon (ARM) but the target is x86-64 (amd64)

## Managing Lima VM

```bash
# Stop the VM
limactl stop

# Start the VM
limactl start

# Delete the VM
limactl delete default

# Show VM info
limactl list

# Show resource usage
limactl shell default df -h
limactl shell default free -h
```

## Complete Workflow Example

```bash
# 1. Start Lima (if not running)
limactl start

# 2. Enter Lima
lima

# 3. Login to registry (first time only)
docker login ghcr.io -u rolandpotthast
# Enter PAT when prompted

# 4. Build SIF
cd ~
apptainer build my-container.sif docker://ghcr.io/org/image:latest

# 5. Exit Lima
exit

# 6. Copy to Mac
limactl copy default:my-container.sif ~/Desktop/

# 7. Transfer to HPC
scp ~/Desktop/my-container.sif user@hpc:/destination/
```

## Tips and Best Practices

- **Keep Lima running:** Start it once and leave it running; it uses minimal resources when idle
- **Build in `~`:** Always build SIF files in Lima's home directory, not macOS-mounted paths
- **Save your PAT securely:** Store it in a password manager; you'll need it for future logins
- **Use specific tags:** Instead of `:latest`, use version tags (`:v1.2.3`) for reproducibility
- **Monitor builds:** Large images can take several minutes; watch the progress
- **Clean up:** Remove old SIF files from Lima to save disk space: `lima rm ~/old-image.sif`

## Resource Limits

Default Lima VM configuration:
- **Memory:** 4GB (adjustable in Lima config)
- **CPUs:** 4 cores
- **Disk:** 100GB

To increase resources, edit Lima's config before first start:
```bash
limactl start --cpus=8 --memory=8
```

## Alternative: Using Docker Desktop

If you have Docker Desktop installed, you can also use it to run Apptainer in a container, but Lima provides a more native Linux experience for Apptainer workflows.

## References

- Lima documentation: https://lima-vm.io/
- Apptainer documentation: https://apptainer.org/docs/
- GitHub PAT guide: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens

# 3
-----

# Building SIF Files with GitHub Actions

This guide explains how to automatically build Apptainer SIF files using GitHub Actions CI/CD infrastructure.

## Why Use GitHub Actions?

Building SIF files with GitHub Actions provides several advantages:

- ✅ **Automated builds:** SIF files are automatically created when Docker images are updated
- ✅ **No local resources needed:** GitHub provides free Linux runners with sufficient memory
- ✅ **Version control:** SIF files are tied to specific image versions
- ✅ **Easy distribution:** Team members can download SIF files without building locally
- ✅ **Consistent environment:** Builds happen in a clean, reproducible environment

## Prerequisites

- GitHub repository with a Dockerfile or Docker image
- GitHub Personal Access Token (PAT) with `packages:read` permission
- Existing Docker image workflow (or combined workflow)

## Overview of the Workflow

The workflow consists of these steps:

1. **Trigger:** Runs after a Docker image is successfully built
2. **Install Apptainer:** Sets up Apptainer on Ubuntu runner
3. **Authenticate:** Uses GitHub PAT to access private container registry
4. **Build SIF:** Pulls Docker image and converts to SIF format
5. **Upload:** Stores SIF file as a downloadable artifact

## Step 1: Create a GitHub Personal Access Token

1. Go to https://github.com/settings/tokens
2. Click **"Generate new token"** → **"Tokens (classic)"**
3. Give it a name: `GHCR_READ_PAT`
4. Set expiration (recommended: 90 days or longer for automation)
5. Check the **`read:packages`** scope
6. Click **"Generate token"**
7. **Copy the token immediately**

## Step 2: Add Token to Repository Secrets

1. Go to your repository on GitHub
2. Navigate to **Settings** → **Secrets and variables** → **Actions**
3. Click **"New repository secret"**
4. Name: `GHCR_READ_PAT`
5. Value: Paste your PAT
6. Click **"Add secret"**

## Step 3: Create the Workflow File

Create `.github/workflows/build-sif.yml` in your repository:

```yaml
name: Export Apptainer SIF (Example 03)

# Trigger after Docker image build completes successfully
on:
  workflow_run:
    workflows: ["Build and publish Docker image (03 plot)"]
    types: [completed]

jobs:
  build-sif:
    # Only run if the Docker build succeeded
    if: ${{ github.event.workflow_run.conclusion == 'success' }}
    runs-on: ubuntu-latest
    
    permissions:
      contents: read
      packages: read
    
    steps:
      - name: Install Apptainer
        run: |
          sudo apt-get update
          sudo apt-get install -y \
            build-essential \
            uuid-dev \
            libgpgme-dev \
            squashfs-tools \
            libseccomp-dev \
            wget \
            pkg-config \
            git \
            cryptsetup
          
          # Download and install Apptainer
          wget https://github.com/apptainer/apptainer/releases/download/v1.3.4/apptainer_1.3.4_amd64.deb
          sudo dpkg -i apptainer_1.3.4_amd64.deb
          
          # Verify installation
          apptainer --version
      
      - name: Pull Docker image and convert to SIF
        env:
          # Set credentials for Apptainer to use
          APPTAINER_DOCKER_USERNAME: ${{ github.actor }}
          APPTAINER_DOCKER_PASSWORD: ${{ secrets.GHCR_READ_PAT }}
        run: |
          # Pull Docker image and convert to SIF
          apptainer pull docker-plot-03.sif \
            docker://ghcr.io/eumetnet-e-ai/docker-plot-03:latest
      
      - name: Upload SIF artifact
        uses: actions/upload-artifact@v4
        with:
          name: docker-plot-03-sif
          path: docker-plot-03.sif
```

## Step 4: Customize for Your Repository

Update these values in the workflow:

```yaml
# Line 4: Match your Docker workflow name
workflows: ["Your Docker Workflow Name"]

# Line 42: Update your image path
apptainer pull your-container.sif \
  docker://ghcr.io/your-org/your-image:latest

# Line 46-48: Update artifact name and path
name: your-container-sif
path: your-container.sif
```

## Understanding the Workflow

### Trigger Configuration

```yaml
on:
  workflow_run:
    workflows: ["Build and publish Docker image (03 plot)"]
    types: [completed]
```

This triggers the workflow **after** your Docker image build completes. The workflow only runs if the Docker build was successful.

### Authentication Method

```yaml
env:
  APPTAINER_DOCKER_USERNAME: ${{ github.actor }}
  APPTAINER_DOCKER_PASSWORD: ${{ secrets.GHCR_READ_PAT }}
```

Apptainer automatically uses these environment variables for authentication. No separate login step needed.

### SIF Creation

```yaml
apptainer pull docker-plot-03.sif \
  docker://ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

This command:
1. Pulls the Docker image from GitHub Container Registry
2. Converts it to SIF format with squashfs compression
3. Saves as `docker-plot-03.sif`

## Step 5: Download the SIF File

After the workflow runs successfully:

1. Go to your repository on GitHub
2. Click **Actions** tab
3. Click on the completed workflow run
4. Scroll down to **Artifacts** section
5. Click on `docker-plot-03-sif` to download

The downloaded file is a zip archive containing your SIF file.

## Step 6: Transfer to HPC

```bash
# Unzip the artifact (on your local machine)
unzip docker-plot-03-sif.zip

# Transfer to HPC
scp docker-plot-03.sif username@hpc-system:/path/to/destination/
```

Example for RCNL:
```bash
scp docker-plot-03.sif rpotthas@rcnl100:/hpc/uwork/rpotthas/apptainer_test/
```

## Alternative: Manual Trigger

If you want to build SIF files on-demand instead of automatically:

```yaml
name: Build Apptainer SIF

on:
  workflow_dispatch:  # Manual trigger
    inputs:
      tag:
        description: 'Docker image tag to build'
        required: true
        default: 'latest'

jobs:
  build-sif:
    runs-on: ubuntu-latest
    steps:
      # ... same steps as above ...
      
      - name: Pull Docker image and convert to SIF
        env:
          APPTAINER_DOCKER_USERNAME: ${{ github.actor }}
          APPTAINER_DOCKER_PASSWORD: ${{ secrets.GHCR_READ_PAT }}
        run: |
          apptainer pull docker-plot-03-${{ inputs.tag }}.sif \
            docker://ghcr.io/eumetnet-e-ai/docker-plot-03:${{ inputs.tag }}
```

To trigger manually:
1. Go to **Actions** tab
2. Select **Build Apptainer SIF** workflow
3. Click **Run workflow**
4. Enter the tag (e.g., `latest` or `v1.0.0`)
5. Click **Run workflow**

## Combined Workflow (Docker + SIF in One)

For simpler setups, you can build both Docker and SIF in a single workflow:

```yaml
name: Build Docker and SIF

on:
  push:
    branches: [main]
  workflow_dispatch:

jobs:
  build-and-export:
    runs-on: ubuntu-latest
    permissions:
      contents: read
      packages: write
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v4
      
      - name: Login to GitHub Container Registry
        uses: docker/login-action@v3
        with:
          registry: ghcr.io
          username: ${{ github.actor }}
          password: ${{ secrets.GITHUB_TOKEN }}
      
      - name: Build and push Docker image
        uses: docker/build-push-action@v5
        with:
          context: .
          push: true
          tags: ghcr.io/${{ github.repository }}:latest
      
      - name: Install Apptainer
        run: |
          sudo apt-get update
          sudo apt-get install -y build-essential uuid-dev libgpgme-dev \
            squashfs-tools libseccomp-dev wget pkg-config git cryptsetup
          wget https://github.com/apptainer/apptainer/releases/download/v1.3.4/apptainer_1.3.4_amd64.deb
          sudo dpkg -i apptainer_1.3.4_amd64.deb
      
      - name: Build SIF from Docker image
        env:
          APPTAINER_DOCKER_USERNAME: ${{ github.actor }}
          APPTAINER_DOCKER_PASSWORD: ${{ secrets.GITHUB_TOKEN }}
        run: |
          apptainer pull my-container.sif \
            docker://ghcr.io/${{ github.repository }}:latest
      
      - name: Upload SIF artifact
        uses: actions/upload-artifact@v4
        with:
          name: my-container-sif
          path: my-container.sif
```

## Troubleshooting

### Issue: "Authentication required" error

**Cause:** PAT is missing or incorrect

**Solution:** 
1. Verify `GHCR_READ_PAT` secret exists in repository settings
2. Check PAT has `read:packages` scope
3. Ensure PAT hasn't expired

### Issue: Workflow doesn't trigger

**Cause:** Workflow name mismatch

**Solution:** Ensure the workflow name in `workflows: ["..."]` exactly matches your Docker workflow name (found in the Docker workflow's `name:` field)

### Issue: "Out of memory" during SIF build

**Cause:** Large Docker images may exceed runner memory

**Solution:** Use GitHub's larger runners (requires paid plan) or simplify your Docker image

### Issue: SIF file is too large

**Cause:** Docker image contains unnecessary files

**Solution:** Optimize your Dockerfile with multi-stage builds and `.dockerignore`

## Best Practices

1. **Use specific tags:** Instead of `:latest`, use version tags (`:v1.0.0`) for reproducibility
2. **Retention policy:** Set artifact retention to 90 days to save storage
3. **Cache dependencies:** Cache Apptainer installation to speed up builds
4. **Test locally first:** Ensure Docker image works before setting up automation
5. **Document versions:** Keep track of which SIF corresponds to which code version

## Artifact Retention

By default, GitHub keeps artifacts for 90 days. You can customize this:

```yaml
- name: Upload SIF artifact
  uses: actions/upload-artifact@v4
  with:
    name: docker-plot-03-sif
    path: docker-plot-03.sif
    retention-days: 30  # Keep for 30 days instead of 90
```

## Advanced: Publish SIF to Release

Instead of artifacts, attach SIF files to GitHub releases:

```yaml
- name: Create Release and Upload SIF
  uses: softprops/action-gh-release@v1
  if: startsWith(github.ref, 'refs/tags/')
  with:
    files: docker-plot-03.sif
  env:
    GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
```

This requires tagging your commits:
```bash
git tag v1.0.0
git push origin v1.0.0
```

## Comparison of Methods

| Method | Pros | Cons |
|--------|------|------|
| **GitHub Actions** | Automated, no local resources, easy distribution | Requires setup, 90-day artifact limit |
| **Local (Mac/Linux)** | Full control, test locally, no automation needed | Requires local setup, manual process |
| **HPC Sandbox** | No SIF needed, works with system limits | Not portable, directory format only |

## Summary

GitHub Actions provides a scalable, automated solution for building SIF files:

1. Set up PAT with `read:packages` permission
2. Add PAT as repository secret
3. Create workflow file with Apptainer installation and build steps
4. Download artifacts from GitHub Actions runs
5. Transfer to HPC system

This method is ideal for teams, continuous integration, and distributing containers to multiple users without requiring them to build locall